#!pip install chromedriver-py==110.0.5481.77
#!pip install selenium

In [14]:
# Load All Necessary Packages

import os


from bs4 import BeautifulSoup
import requests
import time
import re

import pandas as pd
import numpy as np

# set options to be headless, ..
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException     

In [15]:
# open it, go to a website, and get results

edx_course_url = 'https://www.edx.org/search?tab=course#main-content'
edx_program_url = 'https://www.edx.org/search?tab=program#main-content'

# Initialize chrome driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)
wd.implicitly_wait(30)

In [16]:
def loading_status_update_list(tot_count, cat_count, update_freq):
  if (tot_count % update_freq) == 0:
    print('Finishing scrapping', end='')
    print('.....', end='')
    print('.....', end='')
    print(f'Total of {tot_count} courses extracted..... ', end='')
    print(f'Total of {cat_count} catagories extracted.....')
  return 0


def loading_status_update_course(num_course, update_freq):
  if (num_course % update_freq) == 0:
    print('Finishing scrapping', end='')
    print('.....', end='')
    print('.....', end='')
    print(f'Total of {num_course} course.', end='')
  return 0

In [17]:
# Custom function to check if a html element exists

def check_element_exist(xpath):
  try:
    wd.find_element("xpath",xpath)
  except NoSuchElementException:
    return False
  return True

In [18]:
wd.get(edx_course_url)
time.sleep(2)
category_list = ['Computer Science','Data Analysis & Statistics','Language','Engineering','Math']
# category_list =[]
# course_mainpage = BeautifulSoup(wd.page_source, 'html.parser')
# categories = course_mainpage.find_all(id=re.compile('subject-'))
# for cat in categories:
#   category_list.append(cat['value'])

print((category_list))

['Computer Science', 'Data Analysis & Statistics', 'Language', 'Engineering', 'Math']


In [19]:
# Get all course names, provider and url from each category
domain_url = 'https://www.edx.org'
pg_button_xpath = "//button[@class='btn-icon btn-icon-primary btn-icon-md next page-link']"

course_name = []
course_category = []
course_provider = []
course_url = []
num_course = 0
num_cat = 0

In [20]:


for cat in category_list:
  cat_course_url = f"https://www.edx.org/search?subject={cat.replace(' & ', '%20%26%20')}&tab=course&language=English"
  wd.get(cat_course_url)
  num_cat += 1
  if check_element_exist(pg_button_xpath):
    next_pg_button = wd.find_element("xpath", pg_button_xpath)
    # Loop through all pages of course list
    while next_pg_button.is_enabled():
      bs = BeautifulSoup(wd.page_source, 'html.parser')
      course_on_page = bs.find_all('div', class_='discovery-card')
      for course in course_on_page:
        name = course['aria-label']
        #print(name)
        if course.find('img', class_='lazy partner-logo entered loaded') is not None:
          provider = course.find(
              'img', class_='lazy partner-logo entered loaded')['alt']
        else:
          provider = ''
        url_untrim = course.a['href']
        url = domain_url + url_untrim
        course_name.append(name)
        course_category.append(cat)
        course_provider.append(provider)
        course_url.append(url)
        num_course += 1
        loading_status_update_list(num_course, num_cat, 100)
      next_pg_button.click()
      time.sleep(1)

  # To extract course information on last page of course list
  bs = BeautifulSoup(wd.page_source, 'html.parser')
  course_on_page = bs.find_all('div', class_='discovery-card')
  for course in course_on_page:
    name = course['aria-label']
    if course.find('img', class_='lazy partner-logo entered loaded') is not None:
      provider = course.find(
          'img', class_='lazy partner-logo entered loaded')['alt']
    else:
      provider = ''
    url_untrim = course.a['href']
    url = domain_url + url_untrim
    course_name.append(name)
    course_category.append(cat)
    course_provider.append(provider)
    course_url.append(url)
    num_course += 1
    loading_status_update_list(num_course, num_cat, 100)

print(f'Scrapping course list completed! Total of {num_course} courses extracted.')


Finishing scrapping..........Total of 100 courses extracted..... Total of 1 catagories extracted.....
Finishing scrapping..........Total of 200 courses extracted..... Total of 1 catagories extracted.....
Finishing scrapping..........Total of 300 courses extracted..... Total of 1 catagories extracted.....
Finishing scrapping..........Total of 400 courses extracted..... Total of 1 catagories extracted.....
Finishing scrapping..........Total of 500 courses extracted..... Total of 1 catagories extracted.....
Finishing scrapping..........Total of 600 courses extracted..... Total of 1 catagories extracted.....
Finishing scrapping..........Total of 700 courses extracted..... Total of 1 catagories extracted.....
Finishing scrapping..........Total of 800 courses extracted..... Total of 1 catagories extracted.....
Finishing scrapping..........Total of 900 courses extracted..... Total of 2 catagories extracted.....
Finishing scrapping..........Total of 1000 courses extracted..... Total of 2 catag

In [21]:
# Data Matrix
scrap_data = np.array([course_name, course_url, course_provider, course_category]).transpose()
print(len(scrap_data))


2151


In [22]:
scrap_data[0:5]

array([["CS50's Introduction to Computer Science",
        'https://www.edx.org/course/introduction-computer-science-harvardx-cs50x',
        'Harvard University', 'Computer Science'],
       ["CS50's Introduction to Programming with Python",
        'https://www.edx.org/course/cs50s-introduction-to-programming-with-python',
        'Harvard University', 'Computer Science'],
       ["CS50's Introduction to Artificial Intelligence with Python",
        'https://www.edx.org/course/cs50s-introduction-to-artificial-intelligence-with-python',
        'Harvard University', 'Computer Science'],
       ['Python Basics for Data Science',
        'https://www.edx.org/course/python-basics-for-data-science',
        'IBM', 'Computer Science'],
       ['Data Science: R Basics',
        'https://www.edx.org/course/data-science-r-basics',
        'Harvard University', 'Computer Science']], dtype='<U202')

In [24]:
print(np.unique(course_category))

['Computer Science' 'Data Analysis & Statistics' 'Engineering' 'Language'
 'Math']


In [33]:
uniq_url, uniq_url_idx = np.unique(scrap_data[:,1], return_index=True)
uniq_data = scrap_data[uniq_url_idx, :]

mul_cat = []
print(uniq_data)

[['4G Network Essentials'
  'https://www.edx.org/course/4g-network-essentials' '' 'Engineering']
 ['5G Deployment' 'https://www.edx.org/course/5g-deployment' ''
  'Computer Science']
 ['5G Essentials' 'https://www.edx.org/course/5g-essentials'
  'Curtin University' 'Computer Science']
 ...
 ['Guided Project: Predict World Cup Soccer Results with ML'
  'https://www.edx.org/course/world-cup-soccer-game-prediction-using-machine-learning'
  '' 'Computer Science']
 ['Writing, Presenting and Submitting Scientific Papers in English | 英文科技论文写作与学术报告'
  'https://www.edx.org/course/writing-presenting-and-submitting-scientific-paper'
  'Tsinghua University' 'Engineering']
 ['Z/OS REXX Programming'
  'https://www.edx.org/course/zos-rexx-programming' '' 'Computer Science']]


In [34]:


for course in uniq_data:
    c_url = course[1]
    list_of_cat = scrap_data[:, 3][scrap_data[:, 1] == c_url]
    cat_string = ''
    for cat in list_of_cat:
        if cat not in cat_string:
            cat_string += cat + ', '
    cat_string = cat_string[:-2]
    mul_cat.append(cat_string)

print(np.array(mul_cat).shape)
print(uniq_data[:,:3].shape)
#print(uniq_data)
print(mul_cat)

(1189,)
(1189, 3)
['Engineering', 'Computer Science, Engineering', 'Computer Science, Engineering', 'Computer Science', 'Computer Science', 'Computer Science', 'Engineering', 'Engineering', 'Engineering', 'Engineering', 'Engineering', 'Math', 'Math', 'Math', 'Math', 'Engineering', 'Engineering', 'Engineering', 'Computer Science, Engineering', 'Engineering', 'Language', 'Language', 'Computer Science', 'Computer Science', 'Math', 'Computer Science', 'Data Analysis & Statistics, Math', 'Engineering', 'Engineering', 'Engineering', 'Computer Science, Math', 'Math', 'Engineering', 'Computer Science, Data Analysis & Statistics', 'Computer Science', 'Engineering', 'Computer Science', 'Computer Science', 'Engineering', 'Engineering', 'Engineering', 'Computer Science', 'Computer Science, Data Analysis & Statistics', 'Computer Science, Data Analysis & Statistics', 'Computer Science, Data Analysis & Statistics, Engineering', 'Engineering', 'Computer Science, Math', 'Computer Science', 'Computer Sc

In [35]:
# Course List data matrix
data = np.append(uniq_data[:,:3], np.array([mul_cat]).transpose(), axis=1)
edx_pd = pd.DataFrame(data, columns=['Course Name', 'Course URL', 'Course Provider', 'Multi Category'])
edx_pd.to_csv('Edx_Courses_List_new777.csv', index=False, encoding='utf_8_sig')

In [36]:
edx_course_list = pd.read_csv('Edx_Courses_List_new777.csv')
edx_course_list[:30]

,Course Name,Course URL,Course Provider,Multi Category
0,4G Network Essentials,https://www.edx.org/course/4g-network-essentials,NaN,Engineering
1,5G Deployment,https://www.edx.org/course/5g-deployment,NaN,"Computer Science, Engineering"
2,5G Essentials,https://www.edx.org/course/5g-essentials,Curtin University,"Computer Science, Engineering"
3,AI Applications with Watson,https://www.edx.org/course/AI-applications-wit...,NaN,Computer Science
4,AI Chatbots without Programming,https://www.edx.org/course/AI-chatbots-without...,NaN,Computer Science
5,"Data Ethics, AI and Responsible Innovation",https://www.edx.org/course/Data-Ethics-AI-and-...,NaN,Computer Science
6,Introduction to Digital Twin and Applications,https://www.edx.org/course/Introduction-to-Dig...,NaN,Engineering
7,Managing Conflicts on Projects with Cultural a...,https://www.edx.org/course/PMCOMM03,NaN,Engineering
8,Designing Project Information Hubs for Program...,https://www.edx.org/course/PMCOMM04,NaN,Engineering
9,Storytelling That Delivers Program and Project...,https://www.edx.org/course/PMCOMM05,NaN,Engineering


In [13]:
# domain_url = 'https://www.edx.org'
# pg_button_xpath = "//button[@class='btn-icon btn-icon-primary btn-icon-md next page-link']"

# course_name = []
# course_category = []
# course_provider = []
# course_url = []
# num_course = 0
# num_cat = 0


# cat_course_url = f"https://www.edx.org/search?subject='Maths'&tab=course&language=English"
# wd.get(cat_course_url)
# num_cat += 1

# bs = BeautifulSoup(wd.page_source, 'html.parser')
# course_on_page = bs.find_all('div', class_='discovery-card')
# for course in course_on_page:
       
#         url_untrim = course.a['href']
#         print(course.a['href'])
#         url = domain_url + url_untrim
#         print(url)
#         course_name.append(name)
#         course_category.append(cat)
#         course_provider.append(provider)
#         course_url.append(url)
#         num_course += 1

In [14]:
# # Get all course names, provider and url from each category

# domain_url = 'https://www.edx.org'
# pg_button_xpath = "//button[@class='btn next page-link']"

# course_name = []
# course_category = []
# course_provider = []
# course_url = []
# num_course = 0
# num_cat = 0

# for cat in category_list:
#   cat_course_url = f"https://www.edx.org/search?subject={cat.replace(' & ', '%26')}&tab=course&language=English"
#   wd.get(cat_course_url)
#   num_cat += 1
#   if check_element_exist(pg_button_xpath):
#     next_pg_button = wd.find_element("xpath", pg_button_xpath)
#     # Loop through all pages of course list
#     while next_pg_button.is_enabled():
#       bs = BeautifulSoup(wd.page_source, 'html.parser')
#       course_on_page = bs.find_all('div', class_='discovery-card')
#       for course in course_on_page:
#         name = course['aria-label']

#         if course.find('img', class_='lazy partner-logo entered loaded') is not None:
#           provider = course.find(
#               'img', class_='lazy partner-logo entered loaded')['alt']
#         else:
#           provider = ''
#         url_untrim = course.a['href']
#         url = domain_url + url_untrim
#         course_name.append(name)
#         course_category.append(cat)
#         course_provider.append(provider)
#         course_url.append(url)
#         num_course += 1
#         loading_status_update_list(num_course, num_cat, 500)
#       next_pg_button.click()
#       time.sleep(1)
#   # To extract course information on last page of course list
#   bs = BeautifulSoup(wd.page_source, 'html.parser')
#   course_on_page = bs.find_all('div', class_='discovery-card')
#   for course in course_on_page:
#     name = course['aria-label']
#     if course.find('img', class_='lazy partner-logo entered loaded') is not None:
#       provider = course.find(
#           'img', class_='lazy partner-logo entered loaded')['alt']
#     else:
#       provider = ''
#     url_untrim = course.a['href']
#     url = domain_url + url_untrim[:url_untrim.find('?index=product')]
#     course_name.append(name)
#     course_category.append(cat)
#     course_provider.append(provider)
#     course_url.append(url)
#     num_course += 1
#     loading_status_update_list(num_course, num_cat, 500)

# print(
#     f'Scrapping course list completed! Total of {num_course} courses extracted.')


Scrapping course list completed! Total of 120 courses extracted.


In [84]:
# Load Course List of URLs

edx_course_list = pd.read_csv('Edx_Courses_List_new777.csv')
title_list = np.array(edx_course_list['Course Name'])
url_list = np.array(edx_course_list['Course URL'])
provider_list = np.array(edx_course_list['Course Provider'])
cat_list = np.array(edx_course_list['Multi Category'])

# Extraction Individual Course Avaialable Information
target_text = 'already enrolled!'

# Element to search
dur_div1 = 'd-flex align-items-start col-12 pb-4 pb-md-0 col-md-4'
dur_div2 = 'ml-3'
dur_div3 = 'h4 mb-0'

mod_div1 = 'd-flex align-items-center col-12 col-md-4'
mod_div2 = 'd-flex align-items-center col-12 col-md-6'

cost_div1 = 'd-flex align-items-start col-12 pt-4 pt-md-0 col-md-4'
cost_div2 = 'h4 mb-0'

opt_cost_div1 = 'track-comparison-table-wrapper'

sdesc_div1 = 'row no-gutters'
# sdesc_div2 = 'col-md-7 pr-4'
# sdesc_div3 = 'p'

ldesc_div1 = 'mt-2 lead-sm html-data'

enroll_div1 = 'enroll'
frame_ul1 = 'mb-0 pl-3 ml-1'
frame_ul2 = 'pl-3 ml-1 mb-0'
instr_div1 = 'instructor-cards d-flex flex-wrap justify-content-center mx-auto justify-content-md-start'
instr_div2 = 'instructor-cards d-flex flex-wrap justify-content-center mx-auto'


In [85]:
# Initialization
duration = []
commitment = []
# mode = []
cost = []
option = []
short_desc = []
long_desc = []
num_enrol = []
difficulty = []
language = []
prerequisites = []
video_language = []
instructor = []
count_course = 0

In [86]:
len(url_list)
print(url_list)

['https://www.edx.org/course/4g-network-essentials'
 'https://www.edx.org/course/5g-deployment'
 'https://www.edx.org/course/5g-essentials' ...
 'https://www.edx.org/course/world-cup-soccer-game-prediction-using-machine-learning'
 'https://www.edx.org/course/writing-presenting-and-submitting-scientific-paper'
 'https://www.edx.org/course/zos-rexx-programming']


In [87]:
# wd = webdriver.Chrome('chromedriver',options=options)
# wd.implicitly_wait(30)
# wd.get('https://www.edx.org/course/programming-for-everybody-getting-started-with-pyt')
# bs = BeautifulSoup(wd.page_source, 'html.parser')
# enroll_div1='enroll'
# target_text='already enrolled!'
# session_text = bs.find('div', id=enroll_div1)
# print(session_text)
# ss_text=session_text.find('div', class_='d-md-none x-small heading-label text-brand-500')
# if (ss_text is not None) and (target_text in ss_text.text):
#         enrol = ss_text.text.replace(target_text,"")
#         print(enrol)
# else:
#         enrol=''

In [88]:
print(bs.find('ul', class_=frame_ul1))

None


In [89]:

start = 0
batch_size = 100
end = min(start + batch_size, len(url_list))

for url in url_list:
    wd.get(url)
    time.sleep(1)
    bs = BeautifulSoup(wd.page_source, 'html.parser')
    #print(url)
    if bs.find('div', class_=dur_div1) is not None:
        dur1 = bs.find('div', class_=dur_div1)
        dur=dur1.find(
            'div', class_=dur_div2).div.text.replace('Estimated ', '')
        if bs.find('div', class_=dur_div1).div.find('div', class_='small') is not None:
             comt = bs.find('div', class_=dur_div1).div.find(
                 'div', class_='small').text
        else:
             comt = ''
#     elif bs.find('div', class_=dur_div1) is not None:
#         dur = bs.find('div', class_=dur_div1).find(
#             'div', class_=dur_div2).text.replace('Estimated ', '')
#         if bs.find('div', class_=dur_div3).div.find('div', class_='small') is not None:
#              comt = bs.find('div', class_=dur_div3).div.find(
#                  'div', class_='small').text
#         else:
#              comt = ''
    else:
        dur = ''
        comt = ''

  # if bs.find('div', class_=mod_div1) is not None:
    # mod = bs.find('div', class_=mod_div1).div.div.text
  # else:
    # mod = bs.find('div', class_=mod_div2).div.div.text

    if bs.find('div', class_=cost_div1) is not None:
        cst = bs.find('div', class_=cost_div1).find(
            'div', class_=cost_div2).text
        if cst == 'Cost to Enroll':
             opt = bs.find('div', class_=cost_div1).div.find(
                 'div', class_='small').text
        else:
                if bs.find('div', class_=opt_cost_div1) is not None:
                    opt = bs.find('div', class_=opt_cost_div1).find(
                        'p', class_='comparison-item').text
                else:
                    opt = ''
    else:
        cst = 'Free'
        opt = ''

    sdesc1 = bs.find('div', class_=sdesc_div1)
    if sdesc1.div.p is not None:
        sdesc=sdesc1.div.p.text
    else:
         sdesc=''
    ldesc1 = bs.find('div', class_=ldesc_div1)
    if ldesc1 is not None :
         ldesc=ldesc1.text
    else:
         ldesc=''

    session_text = bs.find('div', id=enroll_div1)
    enrol=''
    if session_text is not None :
      ss_text=session_text.find('div', class_='d-md-none x-small heading-label text-brand-500')
      if (ss_text is not None) and (target_text in ss_text.text):
            enrol = ss_text.text.replace(target_text,"")
    
    else:
        enrol = ''

  # Frame information
    vlang=''
    prereq=''
    diff=''
    lang=''
    instr=[]
    if bs.find('ul', class_=frame_ul1) is not None :
        frame1 = bs.find('ul', class_=frame_ul1).find_all('li')
        for item in frame1:
            if 'Level: ' in item.text:
                 diff = item.text.replace('Level: ', '')
            if 'Language: ' in item.text:
                 lang = item.text.replace('Language: ', '')
            if 'Prerequisites: ' in item.text:
                  prereq = item.text.replace('Prerequisites: ', '')
            if 'Video Transcript: ' in item.text:
                 vlang = item.text.replace('Video Transcript: ', '')
    if bs.find('ul', class_=frame_ul2) is not None :
        frame2 = bs.find('ul', class_=frame_ul2).find_all('li')
        for item in frame2:
            if 'Level: ' in item.text:
                  diff = item.text.replace('Level: ', '')
            if 'Language: ' in item.text:
                  lang = item.text.replace('Language: ', '')
            if 'Prerequisites: ' in item.text:
                  prereq = item.text.replace('Prerequisites: ', '')
            if 'Video Transcript: ' in item.text:
                    #print(item.text)
                    vlang = item.text.replace('Video Transcript: ', '')

#   # Instructor information
#     instr_frame = bs.find(
#         'div', class_='instructor-cards d-flex flex-wrap justify-content-center mx-auto justify-content-md-start')
#     if instr_frame is not None:
#            instr_frame = instr_frame.find_all('h3')
#     else:
#            instr_frame = bs.find('div', class_='instructor-cards d-flex flex-wrap justify-content-center mx-auto').find_all('h3')
#     instr = [item.text for item in instr_frame]

    duration.append(dur.strip())
    commitment.append(comt.strip())
  # mode.append(mod)
    cost.append(cst)
    option.append(opt)
    short_desc.append(sdesc.strip())
    long_desc.append(ldesc.strip())
    num_enrol.append(enrol)
    difficulty.append(diff.strip())
    language.append(lang.strip())
    prerequisites.append(prereq)
    video_language.append(vlang.strip())
    #instructor.append(instr)

  # Update status
    count_course += 1
    #loading_status_update_course(count_course, 500)

print('Scrapping course information completed!')

print(len(duration))
print(len(commitment))
# print(len(mode))
print(len(cost))
print(len(option))
print(len(short_desc))
print(len(long_desc))
print(len(num_enrol))
print(len(difficulty))
print(len(language))
print(len(prerequisites))
print(len(video_language))
#print(len(instructor))




Scrapping course information completed!
1189
1189
1189
1189
1189
1189
1189
1189
1189
1189
1189


In [91]:
print((duration))
print((commitment))
# print(len(mode))
print((cost))
print((option))
print((short_desc))
print((long_desc))
print((num_enrol))
print((difficulty))
print((language))
print(len(prerequisites))
print((video_language))
#print((instructor))



['6 weeks', '6 weeks', '7 weeks', '3 weeks', '2 weeks', '7 weeks', '5 weeks', '5 weeks', '4 weeks', '5 weeks', '6 weeks', '7 weeks', '7 weeks', '7 weeks', '7 weeks', '7 weeks', '6 weeks', '5 weeks', '7 weeks', '6 weeks', '10 weeks', '8 weeks', '5 weeks', '4 weeks', '6 weeks', '6 weeks', '6 weeks', '12 weeks', '12 weeks', '12 weeks', '16 weeks', '6 weeks', '7 weeks', '5 weeks', '2 weeks', '6 weeks', '4 weeks', '3 weeks', '4 weeks', '4 weeks', '10 weeks', '3 weeks', '5 weeks', '5 weeks', '6 weeks', '6 weeks', '6 weeks', '6 weeks', '4 weeks', '6 weeks', '6 weeks', '4 weeks', '4 weeks', '5 weeks', '4 weeks', '6 weeks', '5 weeks', '5 weeks', '6 weeks', '11 weeks', '14 weeks', '6 weeks', '6 weeks', '6 weeks', '3 weeks', '5 weeks', '10 weeks', '10 weeks', '6 weeks', '6 weeks', '5 weeks', '16 weeks', '4 weeks', '5 weeks', '1 weeks', '4 weeks', '6 weeks', '5 weeks', '4 weeks', '11 weeks', '8 weeks', '7 weeks', '16 weeks', '5 weeks', '5 weeks', '15 weeks', '13 weeks', '4 weeks', '4 weeks', '4 we

In [100]:
import pandas as pd
data=pd.read_csv("Courses/Edx_Courses_Data11112.csv")

In [122]:
data.head()
difficulty=np.array(difficulty)

In [147]:
df=pd.DataFrame(columns=['Course Name', 'Multi Categories', 'Course URL', 'Course Provider', 'Short Description', 'Long Description', 'Difficulty', 'Duration', 'Commitment', 'Number of Enrolled', 'Cost', 'Alternative Options', 'Language', 'Video Language','Prerequisites'])

In [148]:
df.head()

,Course Name,Multi Categories,Course URL,Course Provider,Short Description,Long Description,Difficulty,Duration,Commitment,Number of Enrolled,Cost,Alternative Options,Language,Video Language,Prerequisites


In [149]:
df['Course Name']=title_list
df['Multi Categories']=cat_list
df['Course URL']=url_list
df['Course Provider']=provider_list
df['Short Description']=short_desc
df['Long Description']=long_desc
df['Difficulty']= difficulty
df['Duration']=duration
df['Commitment']=commitment
df['Number of Enrolled']=num_enrol

df['Cost']=cost
df['Alternative Options']=option
df['Language']=language
df['Video Language']=video_language

df['Prerequisites']=prerequisites

In [152]:
filecount = 11
filepath = 'Courses/'
filename = 'Edx_Courses_Data_New_Final{:d}.csv'.format(filecount)
df.to_csv(filepath+filename, index=False, encoding='utf_8_sig')

In [153]:
df.head()

,Course Name,Multi Categories,Course URL,Course Provider,Short Description,Long Description,Difficulty,Duration,Commitment,Number of Enrolled,Cost,Alternative Options,Language,Video Language,Prerequisites
0,4G Network Essentials,Engineering,https://www.edx.org/course/4g-network-essentials,NaN,Advance your telecom career by learning how 4G...,4G is the cutting-edge network technology that...,Intermediate,6 weeks,3–4 hours per week,,Free,"₹4,007",English,English,"BSc degree (algebra, basic mathematics concep..."
1,5G Deployment,"Computer Science, Engineering",https://www.edx.org/course/5g-deployment,NaN,"Learn about 5G deployment, 5G Radio Access Net...","In this course, you will learn about 5G deploy...",Introductory,6 weeks,4–6 hours per week,,Free,"₹16,273",English,English,There are no pre-requisites for this course. ...
2,5G Essentials,"Computer Science, Engineering",https://www.edx.org/course/5g-essentials,Curtin University,"Learn about 5G Networking, 5G architecture, an...","In this course, you will learn and explore abo...",Introductory,7 weeks,4–6 hours per week,,Free,"₹16,273",English,English,There are no prerequisites for this course.
3,AI Applications with Watson,Computer Science,https://www.edx.org/course/AI-applications-wit...,NaN,Turbocharge your chatbot's IQ and AI capabilit...,Please Note: Learners who successfully complet...,Intermediate,3 weeks,2–4 hours per week,"13,695",Free,"₹8,096",English,English,This course provides an introduction to devel...
4,AI Chatbots without Programming,Computer Science,https://www.edx.org/course/AI-chatbots-without...,NaN,Chatbots are increasingly in demand among glob...,Please Note: Learners who successfully complet...,Introductory,2 weeks,2–4 hours per week,"48,488",Free,"₹8,096",English,English,None


In [155]:
batch_data = np.array([title_list, cat_list, url_list, provider_list, short_desc, long_desc, difficulty, duration, commitment, num_enrol, cost, option, language, video_language,prerequisites ])
batch_pd = pd.DataFrame(batch_data.transpose(), columns=['Course Name', 'Multi Categories', 'Course URL', 'Course Provider', 'Short Description', 'Long Description', 'Difficulty', 'Duration', 'Commitment', 'Number of Enrolled', 'Cost', 'Alternative Options', 'Language', 'Video Language', 'Prerequisites'])

filecount = 1011
filepath = 'Courses/'
filename = 'Edx_Courses_Data_new{:d}.csv'.format(filecount)
batch_pd.to_csv(filepath+filename, index=False, encoding='utf_8_sig')